In [7]:
import xml.etree.ElementTree as ET
import pandas as pd
import json
from catboost import CatBoostClassifier, CatBoostRegressor
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error

Этот код выполняет разбор XML-файла с речевыми данными и извлекает подробную информацию о каждом слове и его характеристиках.

Вот что делает функция `correct_parse_xml_data` по шагам:

1. Загружает и парсит XML-файл.
2. Находит все теги `sentence` — каждое такое вложение рассматривается как предложение.
3. Для каждого предложения перебирает все слова (`word`) и собирает их особенности:
   - Исходный текст слова (`original`).
   - Признак фразового ударения (`phrasal_stress`), если атрибут `nucleus` равен '2'.
   - Лингвистические характеристики: часть речи, форма слова, род, семантика.
4. Определяет длительность паузы после каждого слова (если пауза есть).
5. Сохраняет информацию о позиции слова в предложении и общем количестве слов.
6. Формирует список с данными для всех слов во всех предложениях и возвращает его в виде `DataFrame`.

После этого код выводит статистику:

- Общее число слов.
- Число уникальных предложений.
- Распределение фразовых ударений.
- Данные по паузам — сколько пауз больше 0, средняя длина, диапазон и примеры.


In [5]:
def correct_parse_xml_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    sentences_data = []
    
    for sentence_idx, sentence in enumerate(root.findall('.//sentence')):
        elements = list(sentence)
        words_in_sentence = []
        
        # Собираем все слова и их позиции
        word_elements = [(i, elem) for i, elem in enumerate(elements) if elem.tag == 'word']
        
        for word_pos, word_elem in word_elements:
            original = word_elem.get('original', '')
            has_stress = word_elem.get('nucleus') == '2'
            
            # Лингвистические характеристики
            dictitem = word_elem.find('dictitem')
            if dictitem is not None:
                pos = dictitem.get('subpart_of_speech', '')
                form = dictitem.get('form', '')
                gender = dictitem.get('genesys', '')
                semantics1 = dictitem.get('semantics1', '')
                semantics2 = dictitem.get('semantics2', '')
            else:
                pos = form = gender = semantics1 = semantics2 = ''
            
            # Правильно определяем паузу после слова
            pause_after = -1
            # Ищем следующие элементы после текущего слова
            for next_pos in range(word_pos + 1, len(elements)):
                next_elem = elements[next_pos]
                if next_elem.tag == 'pause':
                    pause_time = next_elem.get('time')
                    if pause_time and pause_time.isdigit():
                        pause_after = int(pause_time)
                    break
                elif next_elem.tag == 'word':
                    # Следующее слово - значит паузы нет
                    break
            
            word_data = {
                'sentence_id': sentence_idx,
                'original': original,
                'position_in_sentence': len(words_in_sentence),
                'total_words_in_sentence': len(word_elements),
                'words_before': len(words_in_sentence),
                'words_after': len(word_elements) - len(words_in_sentence) - 1,
                'has_capital': original and original[0].isupper(),
                'word_length': len(original),
                'part_of_speech': pos,
                'form': form,
                'gender': gender,
                'semantics1': semantics1,
                'semantics2': semantics2,
                'phrasal_stress': has_stress,
                'pause_length': pause_after
            }
            
            words_in_sentence.append(word_data)
        
        sentences_data.extend(words_in_sentence)
    
    return pd.DataFrame(sentences_data)

# Загружаем исправленные данные
df_corrected = correct_parse_xml_data('gogol_utf8_cut.Result.xml')
print(f"Загружено {len(df_corrected)} слов")
print(f"Уникальных предложений: {df_corrected['sentence_id'].nunique()}")

# Статистика
print(f"\nФразовые ударения:")
print(df_corrected['phrasal_stress'].value_counts(normalize=True))

pause_data = df_corrected[df_corrected['pause_length'] > 0]
print(f"\nПаузы (только > 0):")
print(f"Всего пауз: {len(pause_data)}")
if len(pause_data) > 0:
    print(f"Средняя длина: {pause_data['pause_length'].mean():.1f} мс")
    print(f"Мин-Макс: {pause_data['pause_length'].min()}-{pause_data['pause_length'].max()} мс")
    print(f"Примеры пауз: {pause_data['pause_length'].value_counts().head()}")

Загружено 75956 слов
Уникальных предложений: 5001

Фразовые ударения:
phrasal_stress
False    0.763837
True     0.236163
Name: proportion, dtype: float64

Паузы (только > 0):
Всего пауз: 17962
Средняя длина: 383.0 мс
Мин-Макс: 1-1657 мс
Примеры пауз: pause_length
1020    1200
800      310
308      147
297      113
212       96
Name: count, dtype: int64


Этот код выполняет дополнительный анализ данных, извлечённых из XML, и выводит статистику по лингвистическим характеристикам слов, а также примеры слов с паузами.

Подробно:

- Выводит распределение 10 наиболее часто встречающихся частей речи (`part_of_speech`) в датасете.
- Выводит распределение 10 наиболее частых форм слов (`form`).
- Показывает распределение всех значений рода (`gender`).

Далее, если в данных есть слова с паузами больше нуля (`pause_length > 0`):

- Выводит первые 10 примеров слов с паузами, указывая сам текст слова (`original`), длину паузы, позицию слова в предложении и общее количество слов в этом предложении.

Таким образом, код помогает быстро понять, какие части речи, формы и роды преобладают, а также посмотреть конкретные случаи с паузами для дальнейшего анализа или проверки.

In [6]:
# Анализ данных
print("Распределение частей речи:")
print(df_corrected['part_of_speech'].value_counts().head(10))

print("\nРаспределение форм слов:")
print(df_corrected['form'].value_counts().head(10))

print("\nРаспределение родов:")
print(df_corrected['gender'].value_counts())

# Посмотрим на примеры с паузами
if len(pause_data) > 0:
    print("\nПримеры слов с паузами:")
    sample_pauses = pause_data[['original', 'pause_length', 'position_in_sentence', 'total_words_in_sentence']].head(10)
    print(sample_pauses)

Распределение частей речи:
part_of_speech
9     20128
1     15593
6     12607
3      8872
10     6897
2      4495
11     3201
7      1309
0      1069
12      843
Name: count, dtype: int64

Распределение форм слов:
form
0     27445
1      7929
2      3631
5      3559
67     3464
60     2073
6      1773
9      1652
20     1484
10     1460
Name: count, dtype: int64

Распределение родов:
gender
0    52108
1     7019
5     5053
4     4256
6     3860
8     2023
2     1410
3      227
Name: count, dtype: int64

Примеры слов с паузами:
           original  ...  total_words_in_sentence
1              души  ...                        2
3            Гоголь  ...                        2
5            ПЕРВЫЙ  ...                        2
7            первая  ...                        2
14          въехала  ...                       33
19          бричка,  ...                       33
23       холостяки:  ...                       33
25   подполковники,  ...                       33
26  штабс-капитан

Этот код подготавливает данные из предыдущего анализа и обучает две модели машинного обучения для двух задач, затем оценивает их качество и выводит важность признаков.

### Основные шаги:

1. **Предобработка данных:**
   - Создаёт копию исходных данных и заполняет пропуски пустыми строками.
   - Кодирует категориальные переменные (`part_of_speech`, `form`, `gender`, `semantics1`, `semantics2`) с помощью `LabelEncoder` для использования в моделях.

2. **Формирование признаков:**
   - Определяет набор признаков, включающий позицию слова в предложении, длину слова, наличие заглавной буквы и закодированные категориальные признаки.

3. **Подготовка данных для двух задач:**
   - Задача 1 — классификация фразового ударения: признаковая матрица и целевая переменная с меткой ударения.
   - Задача 2 — регрессия длины пауз: берутся только слова, после которых есть паузы, и готовятся соответствующие данные.

4. **Разделение на обучающую и тестовую выборки:** по 80% на обучение и 20% на тестирование, с учётом баланса классов для задачи классификации.

5. **Обучение моделей:**
   - Для классификации ударений используется CatBoostClassifier с балансировкой классов.
   - Для регрессии длины пауз — CatBoostRegressor.

6. **Оценка качества:**
   - Для классификации выводится отчёт с метриками (precision, recall, f1-score и др.).
   - Для регрессии считаются MSE, MAE, RMSE — показатели точности предсказаний пауз.

7. **Анализ важности признаков:**
   - Выводятся топ-10 самых влияющих признаков для каждой из моделей.

В целом, код строит модель, которая учится предсказывать, падает ли фразовое ударение на слово, а также модель, прогнозирующую длительность паузы после слова, используя лингвистические и позиционные признаки.

In [8]:
# Предобработка данных
print("Предобработка данных...")

# Заполняем пропуски
df = df_corrected.copy()
df.fillna('', inplace=True)

# Кодируем категориальные переменные
categorical_columns = ['part_of_speech', 'form', 'gender', 'semantics1', 'semantics2']

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Признаки для моделей
feature_columns = [
    'position_in_sentence', 
    'total_words_in_sentence',
    'words_before', 
    'words_after', 
    'has_capital', 
    'word_length'
] + [col + '_encoded' for col in categorical_columns]

print(f"Используется {len(feature_columns)} признаков:")
print(feature_columns)

# Подготовка данных для двух задач
print("\nПодготовка данных для двух задач...")

# Задача 1: Классификация фразового ударения
X_stress = df[feature_columns]
y_stress = df['phrasal_stress']

# Задача 2: Регрессия для длины пауз (только слова с паузами)
pause_data = df[df['pause_length'] > 0]
X_pause = pause_data[feature_columns]
y_pause = pause_data['pause_length']

print(f"Задача 1 (ударения): {X_stress.shape[0]} примеров")
print(f"Задача 2 (паузы): {X_pause.shape[0]} примеров")

# Разделение на train/test
X_train_stress, X_test_stress, y_train_stress, y_test_stress = train_test_split(
    X_stress, y_stress, test_size=0.2, random_state=42, stratify=y_stress
)

X_train_pause, X_test_pause, y_train_pause, y_test_pause = train_test_split(
    X_pause, y_pause, test_size=0.2, random_state=42
)

print(f"\nРазделение данных:")
print(f"Ударения - train: {X_train_stress.shape[0]}, test: {X_test_stress.shape[0]}")
print(f"Паузы - train: {X_train_pause.shape[0]}, test: {X_test_pause.shape[0]}")

# Обучение моделей
print("\nОбучение моделей...")

# Модель для фразового ударения
stress_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=100,
    class_weights=[1, 3]  # Учитываем несбалансированность классов
)

print("Обучение модели для фразового ударения...")
stress_model.fit(
    X_train_stress, y_train_stress,
    eval_set=(X_test_stress, y_test_stress),
    early_stopping_rounds=50
)

# Модель для длины пауз
pause_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=100
)

print("\nОбучение модели для длины пауз...")
pause_model.fit(
    X_train_pause, y_train_pause,
    eval_set=(X_test_pause, y_test_pause),
    early_stopping_rounds=50
)

# Оценка моделей
print("\nОценка моделей...")

# Оценка классификации
y_pred_stress = stress_model.predict(X_test_stress)
print("Фразовое ударение - отчет классификации:")
print(classification_report(y_test_stress, y_pred_stress))

# Оценка регрессии
y_pred_pause = pause_model.predict(X_test_pause)
mse = mean_squared_error(y_test_pause, y_pred_pause)
mae = mean_absolute_error(y_test_pause, y_pred_pause)

print(f"\nДлина пауз - метрики регрессии:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f} мс")
print(f"RMSE: {np.sqrt(mse):.2f} мс")

# Важность признаков
print("\nВажность признаков для фразового ударения:")
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': stress_model.get_feature_importance()
}).sort_values('importance', ascending=False)
print(feature_importance.head(10))

print("\nВажность признаков для длины пауз:")
feature_importance_pause = pd.DataFrame({
    'feature': feature_columns,
    'importance': pause_model.get_feature_importance()
}).sort_values('importance', ascending=False)
print(feature_importance_pause.head(10))

Предобработка данных...
Используется 11 признаков:
['position_in_sentence', 'total_words_in_sentence', 'words_before', 'words_after', 'has_capital', 'word_length', 'part_of_speech_encoded', 'form_encoded', 'gender_encoded', 'semantics1_encoded', 'semantics2_encoded']

Подготовка данных для двух задач...
Задача 1 (ударения): 75956 примеров
Задача 2 (паузы): 17962 примеров

Разделение данных:
Ударения - train: 60764, test: 15192
Паузы - train: 14369, test: 3593

Обучение моделей...
Обучение модели для фразового ударения...
0:	learn: 0.6416621	test: 0.6424657	best: 0.6424657 (0)	total: 69.5ms	remaining: 1m 9s
100:	learn: 0.4204364	test: 0.4316800	best: 0.4316800 (100)	total: 444ms	remaining: 3.95s
200:	learn: 0.4018501	test: 0.4197831	best: 0.4197534 (197)	total: 764ms	remaining: 3.04s
300:	learn: 0.3898546	test: 0.4149241	best: 0.4147905 (295)	total: 1.09s	remaining: 2.53s
400:	learn: 0.3811757	test: 0.4129008	best: 0.4128290 (385)	total: 1.47s	remaining: 2.2s
500:	learn: 0.3739343	test:

Этот код выполняет следующие действия:

1. **Функция `predict_and_create_json`**:
   - Принимает на вход новый текст (хотя здесь для демонстрации берёт данные из первого предложения набора данных).
   - Для каждого слова в этом предложении подготовляет признаки.
   - С помощью обученной модели классификации предсказывает, есть ли фразовое ударение.
   - С помощью модели регрессии предсказывает длину паузы после слова (при наличии модели).
   - Формирует результат в виде списка слов со своими характеристиками (`content`, `phrasal_stress`, `pause_len`) в удобном JSON-формате.

2. **Демонстрация вывода JSON**:
   - Создаёт пример JSON-результата с предсказаниями по первому предложению.
   - Выводит этот JSON в консоль красиво отформатированным.

3. **Сохранение моделей и кодировщиков**:
   - Сохраняет обученные модели `stress_model` и `pause_model` в файлы для последующего использования.
   - Сохраняет используемые `LabelEncoder`-ы в отдельный файл с помощью `joblib`.

Таким образом, этот код готовит механизм, который получает текст, предсказывает ударения и паузы на уровне слов, возвращает результаты в JSON и даёт возможность сохранять подготовленные модели и кодировщики для практического применения в будущем.

In [9]:
# Функция для предсказания на новых данных и генерации JSON
def predict_and_create_json(text_data, stress_model, pause_model, label_encoders):
    """
    Предсказывает фразовые ударения и длины пауз для нового текста
    и возвращает результат в требуемом JSON-формате
    """
    # Здесь должна быть логика преобразования текста в признаки
    # Для демонстрации используем существующие данные
    
    # Берем случайное предложение из тестовой выборки для демонстрации
    sample_sentence = df[df['sentence_id'] == 0]  # Первое предложение
    
    result = []
    sentence_data = {"words": []}
    
    for _, word_row in sample_sentence.iterrows():
        # Подготавливаем признаки для предсказания
        word_features = pd.DataFrame([word_row[feature_columns]])
        
        # Предсказываем ударение
        stress_pred = stress_model.predict(word_features)[0]
        
        # Предсказываем длину паузы (только если модель обучена)
        pause_pred = -1  # по умолчанию -1 (нет паузы)
        if pause_model:
            pause_pred = pause_model.predict(word_features)[0]
            # Округляем до целых и убеждаемся, что не отрицательное
            pause_pred = max(0, int(round(pause_pred)))
        
        # Добавляем слово в результат
        sentence_data["words"].append({
            "content": word_row['original'],
            "phrasal_stress": bool(stress_pred),
            "pause_len": pause_pred
        })
    
    result.append(sentence_data)
    
    return result

# Генерируем пример JSON-результата
print("\nГенерация примера JSON-результата...")
json_example = predict_and_create_json(None, stress_model, pause_model, label_encoders)

print("Пример JSON-структуры:")
print(json.dumps(json_example, indent=4, ensure_ascii=False))

# Сохраняем модели для будущего использования
print("\nСохранение моделей...")
stress_model.save_model('stress_model.cbm')
pause_model.save_model('pause_model.cbm')

print("Модели сохранены как 'stress_model.cbm' и 'pause_model.cbm'")

# Сохраняем кодировщики
import joblib
joblib.dump(label_encoders, 'label_encoders.pkl')
print("Кодировщики сохранены как 'label_encoders.pkl'")


Генерация примера JSON-результата...
Пример JSON-структуры:
[
    {
        "words": [
            {
                "content": "Мёртвые",
                "phrasal_stress": false,
                "pause_len": 238
            },
            {
                "content": "души",
                "phrasal_stress": true,
                "pause_len": 650
            }
        ]
    }
]

Сохранение моделей...
Модели сохранены как 'stress_model.cbm' и 'pause_model.cbm'
Кодировщики сохранены как 'label_encoders.pkl'


In [ ]:
# Сохраняем JSON в файл
def save_json_result(json_data, filename='result.json'):
    """
    Сохраняет JSON-результат в файл с правильным форматированием
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=4, ensure_ascii=False)
    
    print(f"JSON-результат сохранен в файл: {filename}")

# Сохраняем наш пример
save_json_result(json_example, 'example_result.json')

# Теперь создадим более реалистичный пример - обработаем несколько предложений
def process_multiple_sentences(sentence_ids, stress_model, pause_model, label_encoders, df):
    """
    Обрабатывает несколько предложений и возвращает JSON-результат
    """
    result = []
    
    for sentence_id in sentence_ids:
        sentence_data = df[df['sentence_id'] == sentence_id]
        
        if len(sentence_data) == 0:
            continue
            
        sentence_result = {"words": []}
        
        for _, word_row in sentence_data.iterrows():
            # Подготавливаем признаки для предсказания
            word_features = pd.DataFrame([word_row[feature_columns]])
            
            # Предсказываем ударение
            stress_pred = stress_model.predict(word_features)[0]
            
            # Предсказываем длину паузы
            pause_pred = -1
            if pause_model:
                predicted_pause = pause_model.predict(word_features)[0]
                # Округляем и проверяем, что не отрицательное
                pause_pred = max(0, int(round(predicted_pause)))
            
            # Добавляем слово в результат
            sentence_result["words"].append({
                "content": word_row['original'],
                "phrasal_stress": bool(stress_pred),
                "pause_len": pause_pred
            })
        
        result.append(sentence_result)
    
    return result

# Обрабатываем несколько предложений для демонстрации
print("\nОбработка нескольких предложений для демонстрации...")
sample_sentences = [0, 1, 2, 3]  # Первые 4 предложения
multiple_results = process_multiple_sentences(sample_sentences, stress_model, pause_model, label_encoders, df)

# Сохраняем результат с несколькими предложениями
save_json_result(multiple_results, 'multiple_sentences_result.json')

print("Содержимое файла multiple_sentences_result.json:")
print(json.dumps(multiple_results, indent=4, ensure_ascii=False)[:1000] + "...")  # Показываем начало

# Также создадим функцию для обработки всего датасета (будет долго, но для полноты)
def create_final_submission_json(stress_model, pause_model, label_encoders, df, output_file='final_submission.json'):
    """
    Создает финальный JSON для submission на основе всех данных
    В реальной ситуации здесь нужно обрабатывать тестовые данные
    """
    print(f"Создание финального submission файла...")
    
    # Для демонстрации возьмем только первые 100 предложений
    # В реальном задании нужно обработать всю тестовую выборку
    demo_sentence_ids = df['sentence_id'].unique()[:100]
    
    result = process_multiple_sentences(demo_sentence_ids, stress_model, pause_model, label_encoders, df)
    
    # Сохраняем
    save_json_result(result, output_file)
    
    print(f"Файл {output_file} создан с {len(result)} предложениями")
    return result

# Создаем демо-версию для submission
final_result = create_final_submission_json(stress_model, pause_model, label_encoders, df, 'lab_submission.json')

# Проверяем, что файлы создались
import os
print(f"\nСозданные файлы:")
for file in ['example_result.json', 'multiple_sentences_result.json', 'lab_submission.json']:
    if os.path.exists(file):
        file_size = os.path.getsize(file)
        print(f"✓ {file} ({file_size} bytes)")
    else:
        print(f"✗ {file} - не найден")

JSON-результат сохранен в файл: example_result.json

Обработка нескольких предложений для демонстрации...
JSON-результат сохранен в файл: multiple_sentences_result.json
Содержимое файла multiple_sentences_result.json:
[
    {
        "words": [
            {
                "content": "Мёртвые",
                "phrasal_stress": false,
                "pause_len": 238
            },
            {
                "content": "души",
                "phrasal_stress": true,
                "pause_len": 650
            }
        ]
    },
    {
        "words": [
            {
                "content": "Николай",
                "phrasal_stress": false,
                "pause_len": 160
            },
            {
                "content": "Гоголь",
                "phrasal_stress": true,
                "pause_len": 719
            }
        ]
    },
    {
        "words": [
            {
                "content": "ТОМ",
                "phrasal_stress": false,
                "pause_len

In [ ]:
# Создаем утилитный файл для использования моделей
def create_prediction_script():
    """
    Создает Python-скрипт для предсказания на новых данных
    """
    script_content = '''
import pandas as pd
import json
from catboost import CatBoostClassifier, CatBoostRegressor
import joblib

class ProsodyPredictor:
    def __init__(self, stress_model_path, pause_model_path, encoders_path):
        """Загружает обученные модели и кодировщики"""
        self.stress_model = CatBoostClassifier()
        self.stress_model.load_model(stress_model_path)
        
        self.pause_model = CatBoostRegressor()
        self.pause_model.load_model(pause_model_path)
        
        self.label_encoders = joblib.load(encoders_path)
        
        # Признаки, используемые в моделях
        self.feature_columns = [
            'position_in_sentence', 'total_words_in_sentence', 'words_before', 
            'words_after', 'has_capital', 'word_length', 'part_of_speech_encoded',
            'form_encoded', 'gender_encoded', 'semantics1_encoded', 'semantics2_encoded'
        ]
    
    def predict_sentence(self, sentence_features):
        """
        Предсказывает ударения и паузы для одного предложения
        
        Args:
            sentence_features: DataFrame с признаками для слов предложения
        
        Returns:
            JSON-структура с результатами
        """
        result = {"words": []}
        
        for _, word_row in sentence_features.iterrows():
            # Подготавливаем признаки
            word_features = pd.DataFrame([word_row[self.feature_columns]])
            
            # Предсказываем
            stress_pred = self.stress_model.predict(word_features)[0]
            pause_pred = self.pause_model.predict(word_features)[0]
            pause_pred = max(0, int(round(pause_pred)))
            
            result["words"].append({
                "content": word_row['original'],
                "phrasal_stress": bool(stress_pred),
                "pause_len": pause_pred
            })
        
        return [result]
    
    def save_predictions(self, predictions, filename):
        """Сохраняет предсказания в JSON-файл"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(predictions, f, indent=4, ensure_ascii=False)
        print(f"Предсказания сохранены в {filename}")

# Пример использования:
if __name__ == "__main__":
    # Загружаем модель
    predictor = ProsodyPredictor(
        stress_model_path='stress_model.cbm',
        pause_model_path='pause_model.cbm', 
        encoders_path='label_encoders.pkl'
    )
    
    # Здесь должен быть код для загрузки и предобработки тестовых данных
    # Для демонстрации просто выводим сообщение
    print("Модель загружена. Используйте predictor.predict_sentence() для предсказаний")
'''

    with open('prosody_predictor.py', 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print("Создан файл prosody_predictor.py - утилита для предсказаний")

# Создаем утилитный скрипт
create_prediction_script()

1) пауза стоит после каждого слова. Это проблема. В разметке уже определены места пауз.
Паузы ставить только в конце синтагмы. Тег intonation.
2) Для каждой синтагмы найди ударное слово. Тег nucleus.